In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Paramètres de vol

### Conditions initiales

In [12]:
Vb = 2
Diametre_bouteille = 8.9
Diametre_sortie = 0.9
Masse_fusee_vide = 0.5 # Mo
Cx = 0.1
a = 89 # prévoir de le transformer en radians pour utiliser les fonctions trigo
lr = 22
Po = 10
Veo = 0.65

### Conversion à faire

In [13]:
# Conversion des volumes en mètres cubes
Vb /= 1000
Veo /= 1000
# Conversion de la pression en pascal
Po *= 100000 # press
# Section de la bouteille en m^2
Section_bouteille = (Diametre_bouteille**2)*np.pi/40000
Section_goulot = (Diametre_sortie**2)*np.pi/40000
lr /= 100

### Paramètres environnementaux

In [14]:
g = 9.81 # gravité
r = 998 # densité de l'eau en Kg/m^3
ra = 1.2 # densité de l'air en Kg/m^3
Patm = 101325 # pression atmosphérique

## Constantes théoriques à calculer

### Vitesse initiale (en sortie de rampe)

In [15]:
Ax = (Po*Section_goulot-(Masse_fusee_vide+1000*Veo)*g*np.cos((90-a)*np.pi/180))/(Masse_fusee_vide+1000*Veo) # Accélération suivant x
t = np.sqrt(2*lr/Ax)
Vx = Ax*t # Vitesse au lancement

In [16]:
Ax_g = Ax/g # en G
Vx_kmh = Vx * 3.6 # en km/h

### Force de poussée - calcul du $\beta$

In [17]:
beta = r*(1 - ((Section_goulot/Section_bouteille)**2))

## Calcul des paramètres de vol

### 1ère colonne : Volume air (m^3)

In [18]:
volumes_air = [Vb - Veo]
volumes_air_final = (Po + Patm)*(Vb-Veo)/Patm

# Premiere phase
for i in range(28) :
    volumes_air.append(volumes_air[i] + (Vb-volumes_air[0])/29)
# Fin premiere phase 
volumes_air.append(Vb)
volumes_air.append(Vb)
# Début deuxième phase
for i in range(30,48) :
    volumes_air.append(volumes_air[i] + (volumes_air_final-volumes_air[30])/19)
# Fin deuxième phase
volumes_air.append(volumes_air_final)
# Fin vecteur

### 2eme colonne : Pression air en Pascal

In [19]:
pression_air = list()
for i in range(len(volumes_air)) : 
    pression_air.append(((Po + Patm)*(Vb-Veo)/volumes_air[i])-Patm)

### 4eme colonne : Vitesse d'éjection en m/s

In [20]:
# Phase 1
vitesse = list()
for i in range(30) :
    vitesse.append(np.sqrt(2*pression_air[i]/beta))
# Phase 2
for i in range(20) :
    vitesse.append(np.sqrt(2*pression_air[i+30]/ra))

### 3eme colonne : Temps en secondes

In [22]:
# Phase 1
temps = list()
for i in range(30) : 
    temps.append(((2/3)*volumes_air[i]**1.5 - (2/3)*(Vb-Veo)**1.5)/(Section_goulot*np.sqrt(2*Po*(Vb-Veo)/beta)))
# Phase intermédiaire 1
temps.append((((2/3)*volumes_air[30]**1.5 - (2/3)*(Vb)**1.5)/(Section_goulot*np.sqrt(2*Po*(Vb-Veo)/beta)))+temps[29])
# Phase 2
for i in range(19) : 
    temps.append(temps[30+i]+((volumes_air[31+i]-volumes_air[30+i])/(Section_goulot*((vitesse[31+i]+vitesse[30+i])/2))))
# Phase intermédiaire 2
temps.append(temps[49])
temps.append(temps[50]+0.01)
# Phase 3
for i in range(547) :
    temps.append(temps[i + 51]+ 0.05)

### 5eme colonne : Pousée en Newton

In [23]:
# Phase 1
poussee = list()
for i in range(30) : 
    poussee.append(r*Section_goulot*vitesse[i]**2) 
# Phase 2
for i in range(20) :
    poussee.append(ra*Section_goulot*vitesse[30+i]**2)

### 7eme colonne : Masse de la fusée

In [24]:
# Phase 1
masse_fusee = list()
for i in range(30) :
    masse_fusee.append(Masse_fusee_vide+r*(Vb-volumes_air[i]))
# Phase 2
for i in range(569) :
    masse_fusee.append(Masse_fusee_vide)

### Egaliser toutes les listes à 599 éléments

In [25]:
for i in range(549) :
    poussee.append(0)
    volumes_air.append(0)
    pression_air.append(0)
    vitesse.append(0)

### 6eme 8eme et 9eme colonnes : Inclinaison rampe en degrès - Vitesse de la fusée en m/s - Résistance de l'air en Newton

In [26]:
# Initialisation
inclinaison_rampe=[a]
vitesse_fusee = [Vx]
resistance_air = list()
# Phase 1
for i in range(29) :
    inclinaison_rampe.append(inclinaison_rampe[i]-np.arctan(g*np.cos(inclinaison_rampe[i]*np.pi/180)*(temps[i+1]-temps[i])/vitesse_fusee[i])*180/np.pi)

    resistance_air.append(0.5*ra*Section_bouteille*Cx*(vitesse_fusee[i]**2)) 
    
    vitesse_fusee.append(vitesse_fusee[i]+((poussee[i] - resistance_air[i])/(Masse_fusee_vide + r*(Vb-volumes_air[i+1])) - g*np.sin(inclinaison_rampe[i+1]*np.pi/180))*(temps[i+1]-temps[i]))

resistance_air.append(0.5*ra*Section_bouteille*Cx*(vitesse_fusee[29]**2))

# Phase intermédiaire
inclinaison_rampe.append(inclinaison_rampe[29]-np.arctan(g*np.cos(inclinaison_rampe[29]*np.pi/180)*(temps[30]-temps[29])/vitesse_fusee[29])*180/np.pi)
vitesse_fusee.append(vitesse_fusee[29]+(poussee[30]/Masse_fusee_vide)*(temps[30]-temps[29]))
resistance_air.append(0.5*ra*Section_bouteille*Cx*(vitesse_fusee[30]**2))

# Phase 2
for i in range(19) :
    inclinaison_rampe.append(inclinaison_rampe[i+30]-np.arctan(g*np.cos(inclinaison_rampe[i+30]*np.pi/180)*(temps[i+31]-temps[i+30])/vitesse_fusee[i+30])*180/np.pi)

    vitesse_fusee.append(np.abs(vitesse_fusee[i+30]+((poussee[i+31]-resistance_air[i+30])/Masse_fusee_vide-g*np.sin(inclinaison_rampe[i+31]*np.pi/180))*(temps[i+31]-temps[i+30])))

    resistance_air.append(0.5*ra*Section_bouteille*Cx*(vitesse_fusee[i+31]**2))

# Phase 3
for i in range(549) :
    if vitesse_fusee[-2] < vitesse_fusee[-1] :
        inclinaison_rampe.append(-np.abs(inclinaison_rampe[-1]-np.arctan((g*np.cos(inclinaison_rampe[-1]*np.pi/180)*(temps[49+i]-temps[48+i]))/vitesse_fusee[-1])*180/np.pi))
    else : 
        inclinaison_rampe.append(inclinaison_rampe[-1]-np.arctan((g*np.cos(inclinaison_rampe[-1]*np.pi/180)*(temps[49+i]-temps[48+i]))/vitesse_fusee[-1])*180/np.pi)
    
    vitesse_fusee.append(np.abs(vitesse_fusee[-1]+((poussee[49+i]-resistance_air[48+i])/Masse_fusee_vide -g*np.sin(inclinaison_rampe[-1]*np.pi/180))*(temps[49+i]-temps[48+i])))

    resistance_air.append(0.5*ra*Section_bouteille*Cx*(vitesse_fusee[-1]**2))

### 10eme et 11eme colonnes : X(m) et Y(m)

In [28]:
x = [0]
y = [0]
for i in range(1,599) :
    x.append(x[i-1]+vitesse_fusee[i]*(temps[i]-temps[i-1])*np.cos(inclinaison_rampe[i]*np.pi/180))
    y.append(y[i-1]+vitesse_fusee[i]*(temps[i]-temps[i-1])*np.sin(inclinaison_rampe[i]*np.pi/180))

### 12ème colonne : Accélération en m/s²

In [29]:
acceleration = [0]
for i in range(1,599) :
    if i ==30 or i == 50 : 
        acceleration.append(acceleration[-1]) # mettre un doublon à la frontière
    else : 
        acceleration.append((vitesse_fusee[i]-vitesse_fusee[i-1])/(temps[i]-temps[i-1]))

## Mise en place du tableau final

In [30]:
my_data = np.array([volumes_air,pression_air,temps,vitesse,poussee,masse_fusee,inclinaison_rampe,vitesse_fusee,resistance_air,x,y,acceleration]).T

In [31]:
colonnes = ["Volume d'air","Pression d'air","Temps","Vitesse","Poussée","Masse de la fusée","Inclinaison","Vitesse de la fusée","Résistance de l'air","x(t)","y(t)","Accélération"]

tableau_complet = pd.DataFrame(my_data, columns=colonnes)
tableau_complet

,Volume d'air,Pression d'air,Temps,Vitesse,Poussée,Masse de la fusée,Inclinaison,Vitesse de la fusée,Résistance de l'air,x(t),y(t),Accélération
0,0.001350,1000000.000000,0.000000,44.768489,127.247809,1.148700,89.000000,4.474905,0.007475,0.000000,0.000000,0.000000
1,0.001372,982013.536432,0.007902,44.364049,124.959071,1.126331,88.982677,5.290125,0.010446,0.000742,0.041798,103.160407
2,0.001395,964605.129790,0.015870,43.969064,122.743889,1.103962,88.967647,6.113728,0.013952,0.001620,0.090500,103.373583
3,0.001417,947747.354015,0.023901,43.583161,120.598774,1.081593,88.954344,6.946301,0.018011,0.002638,0.146280,103.663083
4,0.001440,931414.491018,0.031997,43.205987,118.520453,1.059224,88.942390,7.788457,0.022642,0.003802,0.209319,104.030432
...,...,...,...,...,...,...,...,...,...,...,...,...
594,0.000000,0.000000,28.270883,0.000000,0.000000,0.500000,-89.928875,110.905047,4.591174,12.134915,-1730.815201,0.638254
595,0.000000,0.000000,28.320883,0.000000,0.000000,0.500000,-89.929189,110.936693,4.593795,12.141771,-1736.362031,0.632928
596,0.000000,0.000000,28.370883,0.000000,0.000000,0.500000,-89.929502,110.968075,4.596394,12.148597,-1741.910431,0.627644
597,0.000000,0.000000,28.420883,0.000000,0.000000,0.500000,-89.929814,110.999195,4.598972,12.155396,-1747.460386,0.622403
